In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
HISTORY = 6 #indicates how many past datapoints are considered (e.g prediction based on pas 6 hours)
FUTURE = 6  #indicates prediction into future (e.g 1 timestep into the future)
OVERLAP = 1 #indicates how much the data is overlapped

In [3]:
#data = pd.read_csv("../data/finaldata/30min_allData_filtered.csv")
data = pd.read_csv("../data/finaldata/1h_allData_filtered.csv")

In [4]:
data

,time,date,weekday,hour,loc,temp_pi,temp_sbs,temp_blue,hum_pi,hum_sbs,...,wind,wind_speed,wind_direction,cloud_cover,pressure,AQI,sunrise,noon,sunset,daylight
0,2021-03-18 17:00:00+01:00,2021-03-18,3,17,A,25.128571,NaN,4.886313,36.428571,NaN,...,NaN,11.119281,60.945396,62.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
1,2021-03-18 18:00:00+01:00,2021-03-18,3,18,A,24.350000,NaN,4.706314,34.175000,NaN,...,NaN,9.826088,61.557068,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
2,2021-03-18 19:00:00+01:00,2021-03-18,3,19,A,6.480000,NaN,4.456314,49.986667,NaN,...,NaN,9.178235,64.440025,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
3,2021-03-18 20:00:00+01:00,2021-03-18,3,20,A,3.066667,NaN,4.036314,61.100000,NaN,...,NaN,7.235910,84.289406,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
4,2021-03-18 21:00:00+01:00,2021-03-18,3,21,A,2.458333,NaN,3.356314,66.141667,NaN,...,NaN,6.638072,139.398700,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,2021-05-04 18:00:00+02:00,2021-05-04,1,18,B,18.481818,16.0,NaN,37.609091,34.0,...,35.2,NaN,NaN,NaN,1012.0,41.0,2021-05-04 06:04:47.914541+02:00,2021-05-04 13:22:38+02:00,2021-05-04 20:41:16.276717+02:00,0 days 14:36:28.362176
1076,2021-05-04 19:00:00+02:00,2021-05-04,1,19,B,16.550000,NaN,NaN,50.075000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-04 06:04:47.914541+02:00,2021-05-04 13:22:38+02:00,2021-05-04 20:41:16.276717+02:00,0 days 14:36:28.362176
1077,2021-05-04 20:00:00+02:00,2021-05-04,1,20,B,14.775000,NaN,NaN,64.150000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-04 06:04:47.914541+02:00,2021-05-04 13:22:38+02:00,2021-05-04 20:41:16.276717+02:00,0 days 14:36:28.362176
1078,2021-05-04 21:00:00+02:00,2021-05-04,1,21,B,14.375000,NaN,NaN,64.841667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-04 06:04:47.914541+02:00,2021-05-04 13:22:38+02:00,2021-05-04 20:41:16.276717+02:00,0 days 14:36:28.362176


In [5]:
hours = pd.to_datetime(data['time'], utc=True).dt.hour

In [6]:
locs = data['loc']

In [7]:
features = ['temp_pi', 'hum_pi', 'PM1_pi', 'PM2.5_pi', 'PM10_pi']

In [8]:
targets = ['PM2.5_pi', 'PM10_pi']

In [9]:
data = data[features]

In [10]:
features_new = []
for i in range(HISTORY):
    features_new += [f + "_t" + str(i) for f in features]

In [11]:
features_new += targets 
features_new += ['hour', 'loc']

In [12]:
x = math.ceil(data.shape[0]/OVERLAP)
y = len(features_new) 
data_new = np.zeros((x, y))

In [13]:
k = 0
for i in range(HISTORY - 1, data.shape[0] - FUTURE, OVERLAP):
    temp = np.zeros(len(features_new))
    for j in range(HISTORY):
        temp[j*(len(features)):(j+1)*(len(features))] = data.iloc[i - j, :]
    
    temp[len(features)*HISTORY] = data.iloc[i+FUTURE, 3]
    temp[len(features)*HISTORY+1] = data.iloc[i+FUTURE, 4]
    temp[len(features)*HISTORY+2] = hours[i + FUTURE]
    temp[len(features)*HISTORY+3] = 0 if locs[i] == 'A' else 1
    data_new[k,:] = temp
    k += 1

In [14]:
data_new = pd.DataFrame(data_new, columns=features_new)

In [15]:
data_new

,temp_pi_t0,hum_pi_t0,PM1_pi_t0,PM2.5_pi_t0,PM10_pi_t0,temp_pi_t1,hum_pi_t1,PM1_pi_t1,PM2.5_pi_t1,PM10_pi_t1,...,PM10_pi_t4,temp_pi_t5,hum_pi_t5,PM1_pi_t5,PM2.5_pi_t5,PM10_pi_t5,PM2.5_pi,PM10_pi,hour,loc
0,2.200000,67.791667,7.280000,14.491667,21.935833,2.458333,66.141667,6.643333,14.654167,23.088333,...,14.282500,25.128571,36.428571,4.015714,9.323571,14.962143,12.108333,18.262500,3.0,0.0
1,2.075000,69.941667,6.376667,11.823333,17.340833,2.200000,67.791667,7.280000,14.491667,21.935833,...,18.859333,24.350000,34.175000,2.905000,8.352500,14.282500,13.358333,20.365833,4.0,0.0
2,2.050000,71.791667,8.087500,16.170000,24.518333,2.075000,69.941667,6.376667,11.823333,17.340833,...,28.408333,6.480000,49.986667,5.054000,11.749333,18.859333,12.628333,19.224167,5.0,0.0
3,1.983333,69.816667,6.544167,13.952500,21.710833,2.050000,71.791667,8.087500,16.170000,24.518333,...,23.088333,3.066667,61.100000,7.343333,17.540833,28.408333,7.764167,10.910833,6.0,0.0
4,1.758333,69.583333,5.843333,10.430833,15.020833,1.983333,69.816667,6.544167,13.952500,21.710833,...,21.935833,2.458333,66.141667,6.643333,14.654167,23.088333,10.904167,15.890000,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1076,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1078,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [16]:
data_new.to_csv('1h_model_data.csv')